In [ ]:
from util._io import read_config
from collections import namedtuple
from pathlib import Path
import pandas as pd
import numpy as np
import fastsca
from sklearn.metrics.cluster import contingency_matrix,adjusted_rand_score
import matplotlib.pylab as plt
import seaborn as sns
np.random.seed(42)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
experiment = '/projects/experiments/asapp/'
server = Path.home().as_posix()
experiment_home = server+experiment
experiment_config = read_config(experiment_home+'config.yaml')
args = namedtuple('Struct',experiment_config.keys())(*experiment_config.values())

sca = fastsca.FASTSCA()
sca.config = args
sca.initdata()
fn = sca.config.home + sca.config.experiment +sca.config.output + sca.config.sample_id+'/'+sca.config.sample_id

In [ ]:


df_gt = pd.read_csv(fn+'_metadata.csv.gz')
df_tl = pd.read_csv(fn+'_scanpy_label.csv.gz')
df_join = pd.merge(df_tl,df_gt,on='cell',how='left')
print('ARI',adjusted_rand_score(df_join.label_y,df_join.label_x))
df_scmatch = pd.DataFrame(contingency_matrix(df_join.label_x,df_join.label_y))
df_scmatch = df_scmatch.div(df_scmatch.sum(axis=1), axis=0)
sns.distplot(df_scmatch.max(1).values,hist=False)
plt.xlabel('Proportion of cell type')
plt.ylabel('Leiden group count')

print('clust_max',df_scmatch.max(1).sum()/df_scmatch.shape[0])


In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

df = pd.read_csv(fn+'_cluster_trace.csv.gz')

for i in range(df.shape[0]):
    sns.distplot(df.iloc[i,:],label=i,hist=False)
plt.xlabel('proportion of clust max')
plt.legend()


In [ ]:
dfv = pd.read_csv(fn+'_cluster_val_trace.csv.gz')
dfv

In [ ]:
import _topics_util as tpu 
import matplotlib.pylab as plt
import seaborn as sns

dfh = pd.read_csv(fn+'_10_beta.csv.gz')

dfh.columns = sca.data.cols
df_top = tpu.get_topic_top_genes(dfh,10)

df_top = df_top.pivot(index='Topic',columns='Gene',values='Proportion')
sns.clustermap(df_top.T,cmap='viridis')

In [ ]:

df = pd.read_csv(fn+'_cluster_val_trace.csv.gz')
df

In [ ]:
import _topics_util as tpu 
import matplotlib.pylab as plt
import seaborn as sns

dfh = pd.read_csv(fn+'_10_beta.csv.gz')

dfh.columns = sca.data.cols
df_top = tpu.get_topic_top_genes(dfh,20)

df_top = df_top.pivot(index='Topic',columns='Gene',values='Proportion')
sns.clustermap(df_top.T,cmap='viridis')

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

df = pd.read_csv(fn+'_cluster_trace_NMF.csv.gz')

for i in range(df.shape[0]):
    sns.distplot(df.iloc[i,:],label=i,hist=False)
plt.xlabel('proportion of clust max')
plt.legend()

In [ ]:
dfv = pd.read_csv(fn+'_cluster_val_trace_NMF.csv.gz')
dfv